In [1]:
! pip install qiskit

In [2]:
! pip install qiskit-machine-learning

In [3]:
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from qiskit import *

# External imports
from pylab import cm
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.visualization import circuit_drawer
#from qiskit.algorithms.optimizers import SPSA

from qiskit.circuit.library import PauliFeatureMap
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import TrainableFidelityQuantumKernel
from qiskit_machine_learning.kernels.algorithms import QuantumKernelTrainer
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.datasets import ad_hoc_data

#from qiskit.utils import QuantumInstance
#from qiskit_machine_learning.kernels import QuantumKernel

import random
import time
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

/opt/conda/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/conda/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [26]:
df = pd.read_csv('Thesis IBM Dataset.csv')
df.head()

,Date,Time,Temperature(C),Feel likes(C),Chance of Prec.(%),Humidity(%),Visibity(km),Pressure(hPa),State,Real State,Rain,Cloudy,Sunny,Fog
0,27-06-2023,08:00,27,32,21,87,3.0,1003,Cloudy,Cloudy,0.0,1,0,0
1,27-06-2023,09:00,27,32,22,86,9.0,1003,Cloudy,Cloudy,0.0,1,0,0
2,27-06-2023,10:00,28,34,23,90,3.0,1003,Cloudy,Rain,1.0,1,0,0
3,27-06-2023,11:00,28,33,30,85,9.0,1003,Isolated Thunderstroms,Rain,1.0,1,0,0
4,27-06-2023,12:00,27,32,53,90,9.0,1003,Rain,Sunny,0.0,0,1,0


In [27]:
df.drop(columns=['Date'], inplace=True)
#df.drop(columns=['Temperature(C)'], inplace=True)
df

,Time,Temperature(C),Feel likes(C),Chance of Prec.(%),Humidity(%),Visibity(km),Pressure(hPa),State,Real State,Rain,Cloudy,Sunny,Fog
0,08:00,27,32,21,87,3.0,1003,Cloudy,Cloudy,0.0,1,0,0
1,09:00,27,32,22,86,9.0,1003,Cloudy,Cloudy,0.0,1,0,0
2,10:00,28,34,23,90,3.0,1003,Cloudy,Rain,1.0,1,0,0
3,11:00,28,33,30,85,9.0,1003,Isolated Thunderstroms,Rain,1.0,1,0,0
4,12:00,27,32,53,90,9.0,1003,Rain,Sunny,0.0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4499,19:00,20,20,1,77,8.0,1014,Clear,Clear,0.0,0,1,0
4500,20:00,19,19,1,80,14.0,1015,Clear,Clear,0.0,0,1,0
4501,21:00,19,19,2,82,14.0,1015,Clear,Clear,0.0,0,1,0
4502,22:00,18,18,2,83,14.0,1015,Clear,Clear,0.0,0,1,0


In [28]:
df.dropna(inplace=True)
df

,Time,Temperature(C),Feel likes(C),Chance of Prec.(%),Humidity(%),Visibity(km),Pressure(hPa),State,Real State,Rain,Cloudy,Sunny,Fog
0,08:00,27,32,21,87,3.0,1003,Cloudy,Cloudy,0.0,1,0,0
1,09:00,27,32,22,86,9.0,1003,Cloudy,Cloudy,0.0,1,0,0
2,10:00,28,34,23,90,3.0,1003,Cloudy,Rain,1.0,1,0,0
3,11:00,28,33,30,85,9.0,1003,Isolated Thunderstroms,Rain,1.0,1,0,0
4,12:00,27,32,53,90,9.0,1003,Rain,Sunny,0.0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4499,19:00,20,20,1,77,8.0,1014,Clear,Clear,0.0,0,1,0
4500,20:00,19,19,1,80,14.0,1015,Clear,Clear,0.0,0,1,0
4501,21:00,19,19,2,82,14.0,1015,Clear,Clear,0.0,0,1,0
4502,22:00,18,18,2,83,14.0,1015,Clear,Clear,0.0,0,1,0


In [31]:
Final = []
for row in df['Real State']:
        if row=='Rain' or row=='Light Rain' or row=='Heavy Rain' or row=='Heavy rain' or row=='Showers' or row=='showers' or row=='Heavy Thunderstroms' :    Final.append(0)
        elif row=='Cloudy' or row=='Mostly Cloudy' or row=='Mostly cloudy' or row=='cloudy' or row=='Cloudy ':  Final.append(1)
        elif row=='Sunny' or row=='Clear' or row=='Sunny ' or row=='Suuny':  Final.append(2)
        elif row =='Fog' or row=='fog':  Final.append(3)
        elif row=='Thunderstorms' or row=='Thunderstroms' or row=='Isolated Thunderstroms' or row=='Isolated Thunderstorms ' or row=='Isolated Thunderstroms ' or row=='scattered Thunderstorms' or row=='scattered Thunderstroms' or row=='Scattered Thunderstroms' :  Final.append(0)

        else:  Final.append(4)

In [32]:
df['Final'] = Final

    
df

,Time,Temperature(C),Feel likes(C),Chance of Prec.(%),Humidity(%),Visibity(km),Pressure(hPa),State,Real State,Rain,Cloudy,Sunny,Fog,Final
0,08:00,27,32,21,87,3.0,1003,Cloudy,Cloudy,0.0,1,0,0,1
1,09:00,27,32,22,86,9.0,1003,Cloudy,Cloudy,0.0,1,0,0,1
2,10:00,28,34,23,90,3.0,1003,Cloudy,Rain,1.0,1,0,0,0
3,11:00,28,33,30,85,9.0,1003,Isolated Thunderstroms,Rain,1.0,1,0,0,0
4,12:00,27,32,53,90,9.0,1003,Rain,Sunny,0.0,0,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4499,19:00,20,20,1,77,8.0,1014,Clear,Clear,0.0,0,1,0,2
4500,20:00,19,19,1,80,14.0,1015,Clear,Clear,0.0,0,1,0,2
4501,21:00,19,19,2,82,14.0,1015,Clear,Clear,0.0,0,1,0,2
4502,22:00,18,18,2,83,14.0,1015,Clear,Clear,0.0,0,1,0,2


In [33]:
df.drop(columns=['Time', 'State','Real State'], inplace=True)
df

,Temperature(C),Feel likes(C),Chance of Prec.(%),Humidity(%),Visibity(km),Pressure(hPa),Rain,Cloudy,Sunny,Fog,Final
0,27,32,21,87,3.0,1003,0.0,1,0,0,1
1,27,32,22,86,9.0,1003,0.0,1,0,0,1
2,28,34,23,90,3.0,1003,1.0,1,0,0,0
3,28,33,30,85,9.0,1003,1.0,1,0,0,0
4,27,32,53,90,9.0,1003,0.0,0,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...
4499,20,20,1,77,8.0,1014,0.0,0,1,0,2
4500,19,19,1,80,14.0,1015,0.0,0,1,0,2
4501,19,19,2,82,14.0,1015,0.0,0,1,0,2
4502,18,18,2,83,14.0,1015,0.0,0,1,0,2


In [34]:
df.drop(columns=['Rain','Cloudy','Sunny','Fog'], inplace=True)


In [9]:
from qiskit import *

In [10]:
IBMQ.load_account()

Traceback (most recent call last):
  Cell In[10], line 1
    IBMQ.load_account()
NameError: name 'IBMQ' is not defined

Use %tb to get the full traceback.


In [35]:
df

,Temperature(C),Feel likes(C),Chance of Prec.(%),Humidity(%),Visibity(km),Pressure(hPa),Final
0,27,32,21,87,3.0,1003,1
1,27,32,22,86,9.0,1003,1
2,28,34,23,90,3.0,1003,0
3,28,33,30,85,9.0,1003,0
4,27,32,53,90,9.0,1003,2
...,...,...,...,...,...,...,...
4499,20,20,1,77,8.0,1014,2
4500,19,19,1,80,14.0,1015,2
4501,19,19,2,82,14.0,1015,2
4502,18,18,2,83,14.0,1015,2


In [ ]:
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider.backends()

In [20]:
#df.drop(columns=['Visibity(km)','Dew Point','Pressure(hPa)'], inplace=True)
X = df.drop(['Final'], axis='columns')
y = df.Final
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [21]:
X

,Temperature(C),Feels Like(C),Chance of Prec.(%),Humidity(%),Visibity(km),Dew Point,Pressure(hPa)
0,26,31,62,91,4.0,25,1003
1,27,33,66,88,4.0,25,1004
2,27,33,66,88,4.0,25,1004
3,28,33,66,88,4.0,25,1004
4,29,36,65,93,1.6,28,1005
...,...,...,...,...,...,...,...
4499,20,20,1,77,8.0,15,1014
4500,19,19,1,80,14.0,13,1015
4501,19,19,2,82,14.0,13,1015
4502,18,18,2,83,14.0,12,1015


In [22]:
y

0       1
1       1
2       0
3       0
4       2
       ..
4499    2
4500    2
4501    2
4502    2
4503    2
Name: Final, Length: 4504, dtype: int64

In [ ]:
df

In [ ]:
print("Using 3 features Price, Open, High: ")
sz = []
svm_acc = []
dt_acc = []
rf_acc = []
for data_size in range(50, 300, 50):
    print("for size", data_size)
    sz.append(data_size)
    x = df.iloc[:data_size, 0:-1]
    y = df.iloc[:data_size, -1]
    # spltting the dataset into train and test set
    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 31)
    from sklearn import svm
    svc_clf = svm.SVC(random_state = 7)
    svc_clf.fit(x_train, y_train)
    svc_score = svc_clf.score(x_test, y_test)
    print(f"svm classification score: {svc_score}")
    svm_acc.append(svc_score)
    import qiskit
    import matplotlib.pyplot as plt
    import numpy as np

    from sklearn.svm import SVC
    from sklearn.cluster import SpectralClustering
    from sklearn.metrics import normalized_mutual_info_score
'''
    from qiskit import BasicAer
    from qiskit.circuit.library import ZZFeatureMap
    from qiskit.utils import QuantumInstance, algorithm_globals
    from qiskit_machine_learning.algorithms import QSVC
    from qiskit_machine_learning.kernels import QuantumKernel
    from qiskit_machine_learning.datasets import ad_hoc_data
'''
    clf = DecisionTreeClassifier(random_state=42)

    # Train the classifier on the training data
    clf.fit(x_train_, y_train)

    # Make predictions on the testing data
    labels_pred = clf.predict(x_test_)

    dt_accuracy = accuracy_score(y_test_, labels_pred)
    dt_score = metrics.balanced_accuracy_score(y_true=y_test_, y_pred=labels_pred)
    dt_f1_score = f1_score(y_test_, labels_pred)

    print(f"dt classification test score: {dt_f1_score}")
    dt_acc.append(dt_f1_score)
    
    rfc = RandomForestClassifier(n_estimators=100, random_state=42)

    # Train the classifier on the training data
    rfc.fit(X_train, y_train)


    # Make predictions on the testing data
    labels_pred = rfc.predict(x_test_)

    rf_accuracy = accuracy_score(y_test_, labels_pred)
    rf_score = metrics.balanced_accuracy_score(y_true=y_test_, y_pred=labels_pred)
    rf_f1_score = f1_score(y_test_, labels_pred)
 
    print(f"rf classification test score: {rf_f1_score}")
    rf_acc.append(rf_f1_score)

In [ ]:
print(svm_acc)
print(dt_acc)
print(rf_acc)

In [ ]:
sz = [50,100,150,200,250]

import random
total = 5
for cnt in range(total):
    print("current iteration: ", cnt+1)
    for idx in range(len(sz)):
        data_size = sz[idx]
        print("for size", data_size)
        st = random.randint(0, 300-data_size)
        x = df.iloc[st:st+data_size, 0:-1]
        y = df.iloc[st:st+data_size, -1]
        # spltting the dataset into train and test set
        from sklearn.model_selection import train_test_split
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 31)
        from sklearn import svm
        svc_clf = svm.SVC(random_state = 7)
        svc_clf.fit(x_train, y_train)
        svc_score = svc_clf.score(x_test, y_test)
        print(f"svm classification score: {svc_score}")
        svm_acc[idx] = svm_acc[idx] + svc_score
        import qiskit
        import matplotlib.pyplot as plt
        import numpy as np

        from sklearn.svm import SVC
        from sklearn.cluster import SpectralClustering
        from sklearn.metrics import normalized_mutual_info_score

        from qiskit import BasicAer
        from qiskit.circuit.library import ZZFeatureMap
        from qiskit.utils import QuantumInstance, algorithm_globals
        from qiskit_machine_learning.algorithms import QSVC
        from qiskit_machine_learning.kernels import QuantumKernel
        from qiskit_machine_learning.datasets import ad_hoc_data

        clf = DecisionTreeClassifier(random_state=42)

        # Train the classifier on the training data
        clf.fit(x_train_, y_train)

        # Make predictions on the testing data
        labels_pred = clf.predict(x_test_)

        dt_accuracy = accuracy_score(y_test_, labels_pred)
        dt_score = metrics.balanced_accuracy_score(y_true=y_test_, y_pred=labels_pred)
        dt_f1_score = f1_score(y_test_, labels_pred)

        print(f"dt classification test score: {dt_f1_score}")

        #print(f"qsvm classification test score: {dse_score}")
        dt_acc[idx] = dt_acc[idx] + dt_f1_score
        
        rfc = RandomForestClassifier(n_estimators=100, random_state=42)

        # Train the classifier on the training data
        rfc.fit(X_train, y_train)


        # Make predictions on the testing data
        labels_pred = rfc.predict(x_test_)

        rf_accuracy = accuracy_score(y_test_, labels_pred)
        rf_score = metrics.balanced_accuracy_score(y_true=y_test_, y_pred=labels_pred)
        rf_f1_score = f1_score(y_test_, labels_pred)

        print(f"rf classification test score: {rf_f1_score}")
        rf_acc[idx] = rf_acc[idx] + rf_f1_score
    print("=============================================")

In [ ]:
print(svm_acc, dt_acc, rf_acc)

In [ ]:
for idx in range(len(sz)):
    print(idx, svm_acc[idx]/6, '\t\t', dt_acc[idx]/6, '\t\t', rf_acc[idx]/6
         )

In [23]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the dataset
'''
data = pd.read_csv('Thesis IBM Dataset.csv')

# Split the dataset into features and labels
X = data.drop(columns=['Date','Time','State','Real State','Rain'])
y = data['Rain']
'''

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Pauli Decision Tree classifier
pauli_tree = PauliTreeClassifier()
pauli_tree.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = pauli_tree.predict(X_test)

# Calculate the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy Score:", accuracy)


Traceback (most recent call last):
  Cell In[23], line 19
    pauli_tree = PauliTreeClassifier()
NameError: name 'PauliTreeClassifier' is not defined

Use %tb to get the full traceback.


In [36]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels


class PauliDecisionTreeClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y):
        # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        # Store the classes seen during fit
        self.classes_ = unique_labels(y)

        # Convert target labels to binary encoding
        self.label_map = {label: idx for idx, label in enumerate(self.classes_)}
        y_encoded = np.array([self.label_map[label] for label in y])

        # Fit the classical decision tree
        self.tree = DecisionTreeClassifier(max_depth=self.max_depth)
        self.tree.fit(X, y_encoded)

        return self

    def predict(self, X):
        # Check is fit had been called
        check_is_fitted(self)
        # Input validation
        X = check_array(X)

        # Predict the class labels
        y_pred_encoded = self.tree.predict(X)

        # Decode the predicted labels
        y_pred = np.array([list(self.label_map.keys())[list(self.label_map.values()).index(idx)]
                           for idx in y_pred_encoded])

        return y_pred


# Example usage:
if __name__ == "__main__":
    # Load the dataset
    '''
    
    data = pd.read_csv('Thesis MCS Final Dataset.csv')
    data.dropna(inplace=True)
    # Split the dataset into features and labels
    X = data.drop(columns=['Date','Time','Predicted_State','Real State','Rain','Cloudy','Sunny','Fog'])
    y = data['Rain']
    '''

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize and train the Pauli Decision Tree classifier
    pauli_tree = PauliDecisionTreeClassifier()
    pauli_tree.fit(X_train, y_train)

    # Predict the labels for the test set
    y_pred = pauli_tree.predict(X_test)

    # Calculate the accuracy score
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy Score:", accuracy)


Accuracy Score: 0.6315205327413984


In [ ]:
X

In [ ]:
y

In [37]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import resample

class PauliRandomForest:
    def __init__(self, n_estimators=100, max_depth=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.estimators = []

    def fit(self, X, y):
        for _ in range(self.n_estimators):
            # Bootstrap sampling with replacement
            X_boot, y_boot = resample(X, y, replace=True, random_state=None)

            # Train a decision tree on the bootstrapped sample
            tree = DecisionTreeClassifier(max_depth=self.max_depth)
            tree.fit(X_boot, y_boot)
            self.estimators.append(tree)

    def predict(self, X):
        # Make predictions for each tree and aggregate results
        predictions = np.zeros((len(X), len(self.estimators)), dtype=int)
        for i, tree in enumerate(self.estimators):
            predictions[:, i] = tree.predict(X)

        # Use majority voting to determine final prediction
        final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=predictions)
        return final_predictions

# Example usage:
if __name__ == "__main__":
    # Load the dataset
    data = pd.read_csv('Thesis IBM Dataset.csv')
    data.dropna(inplace=True)
    '''
    Rain = []
    for row in df['Rain']:
        if row=='Rain' or row=='Light Rain' or row=='Heavy Rain' or row=='Heavy rain' or row=='Showers' or row=='showers' or row=='Heavy Thunderstroms' :    Rain.append(1)
        
        elif row=='Thunderstorms' or row=='Thunderstroms' or row=='Isolated Thunderstroms' or row=='Isolated Thunderstorms ' or row=='Isolated Thunderstroms ' or row=='scattered Thunderstorms' or row=='scattered Thunderstroms' or row=='Scattered Thunderstroms' :  Rain.append(1)

        else:  Rain.append(0)
        
    data['Rain'] = Rain
    
    # Split the dataset into features and labels
    X = data.drop(columns=['Date','Time','State','Real State','Fog'])
    y = data['Fog']
    '''

    # Initialize and train the Pauli Decision Tree classifier
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize and train the Pauli Random Forest classifier
    pauli_rf = PauliRandomForest(n_estimators=100, max_depth=None)
    pauli_rf.fit(X_train, y_train)

    # Predict the labels for the test set
    y_pred = pauli_rf.predict(X_test)

    # Calculate the accuracy score
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy Score:", accuracy)

Accuracy Score: 0.7169811320754716


In [ ]:
X

In [ ]:
y

In [ ]:
import numpy as np

class QuantumSupportVectorClassifier:
    def __init__(self, n_qubits=2, learning_rate=0.1, epochs=100):
        self.n_qubits = n_qubits
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.theta = np.random.rand(n_qubits)  # Initialize random parameters

    def quantum_kernel(self, x1, x2):
        # Define quantum feature map
        phi_x1 = self.quantum_feature_map(x1)
        phi_x2 = self.quantum_feature_map(x2)
        
        # Calculate quantum inner product
        return np.dot(phi_x1, phi_x2)

    def quantum_feature_map(self, x):
        # Implement quantum feature map
        # This function should encode classical data into quantum states
        pass

    def train(self, X, y):
        for epoch in range(self.epochs):
            for i in range(len(X)):
                # Predict the label for the current sample
                phi_x = self.quantum_feature_map(X[i])
                decision = np.dot(self.theta, phi_x)
                predicted_label = np.sign(decision)

                # Update parameters based on the classification error
                if predicted_label != y[i]:
                    self.theta += self.learning_rate * y[i] * phi_x

    def predict(self, X_test):
        predictions = []
        for i in range(len(X_test)):
            # Calculate the quantum feature map for the test sample
            phi_x = self.quantum_feature_map(X_test[i])

            # Calculate the decision function
            decision = np.dot(self.theta, phi_x)

            # Classify based on the sign of the decision function
            predictions.append(np.sign(decision))

        return predictions



In [ ]:
import numpy as np



# Example usage:
if __name__ == "__main__":
    # Generate synthetic dataset
    data = pd.read_csv('Thesis IBM Dataset.csv')
    data.dropna(inplace=True)
    '''
    Rain = []
    for row in df['Rain']:
        if row=='Rain' or row=='Light Rain' or row=='Heavy Rain' or row=='Heavy rain' or row=='Showers' or row=='showers' or row=='Heavy Thunderstroms' :    Rain.append(1)
        
        elif row=='Thunderstorms' or row=='Thunderstroms' or row=='Isolated Thunderstroms' or row=='Isolated Thunderstorms ' or row=='Isolated Thunderstroms ' or row=='scattered Thunderstorms' or row=='scattered Thunderstroms' or row=='Scattered Thunderstroms' :  Rain.append(1)

        else:  Rain.append(0)
        
    data['Rain'] = Rain
    '''
    # Split the dataset into features and labels
    for col in data.columns:
        if (col == 'Rain'):
       
          data[col] = np.where(data[col] == 0.0, 0, 1)

    X = data.drop(columns=['Date','Time','State','Real State','Fog'])
    y = data['Fog']

    # Initialize and train the Pauli Decision Tree classifier
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    # Initialize and train the Quantum SVM classifier
    qsvc = QuantumSupportVectorClassifier(n_qubits=2, epochs=100)

    qsvc.train(X_train, y_train)

    # Predict the labels for the test set
    y_pred = qsvm.predict(X_test)

    print("Predictions:", y_pred)


In [40]:
! pip install pennylane

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 61.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 65.8 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 1.0 MB/s eta 0:00:00:00:01


In [45]:
import pennylane as qml
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Define a quantum circuit as a feature map
def quantum_feature_map(x, params):
    qml.templates.BasicEntanglerLayers(x, wires=range(len(x)))
    for i in range(len(x)):
        qml.RY(x[i], wires=i)
    for i in range(len(x) - 1):
        qml.CNOT(wires=[i, i + 1])
    for i in range(len(x)):
        qml.RY(params[i], wires=i)

# Initialize the quantum device
dev = qml.device("default.qubit", wires=7)

# Define a quantum circuit function
@qml.qnode(dev)
def quantum_circuit(x, params):
    quantum_feature_map(x, params)
    return [qml.expval(qml.PauliZ(i)) for i in range(len(x))]

# Convert features to quantum state
def convert_to_quantum_features(X, params):
    quantum_features = []
    for i in range(len(X)):
        result = quantum_circuit(X[i], params)
        quantum_features.append(result)
    return np.array(quantum_features)


# Load the CSV dataset
data = pd.read_csv('Thesis IBM Dataset.csv')

# Split features and target variable
for col in data.columns:
        if (col == 'Rain'):
       
          data[col] = np.where(data[col] == 0.0, 0, 1)

X = data.drop(columns=['Date','Time','State','Real State','Rain','Cloudy','Sunny','Fog'])
y = data['Rain']

# Split dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize decision tree classifier
clf = DecisionTreeClassifier()

# Initialize parameters for quantum circuit
params = np.random.rand(len(X_train[0]))

# Convert training data to quantum features
X_train_quantum = convert_to_quantum_features(X_train.values, params)

# Train the decision tree classifier
clf.fit(X_train_quantum, y_train)

# Convert validation data to quantum features
X_val_quantum = convert_to_quantum_features(X_val.values, params)

# Predict on the validation set
y_pred = clf.predict(X_val_quantum)

# Evaluate accuracy
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy:", accuracy)


Traceback (most recent call last):
  Cell In[45], line 55
    params = np.random.rand(len(X_train[0]))
  File /opt/conda/lib/python3.10/site-packages/pandas/core/frame.py:4090 in __getitem__
    indexer = self.columns.get_loc(key)
  File /opt/conda/lib/python3.10/site-packages/pandas/core/indexes/base.py:3809 in get_loc
    raise KeyError(key) from err
KeyError: 0

Use %tb to get the full traceback.


In [46]:
# Generate synthetic dataset
np.random.seed(0)
num_samples = 1000
num_features = 7
X_synthetic = np.random.rand(num_samples, num_features)
y_synthetic = np.random.randint(0, 2, size=num_samples)

# Split synthetic dataset into training and validation sets
X_train_synthetic, X_val_synthetic, y_train_synthetic, y_val_synthetic = train_test_split(X_synthetic, y_synthetic, test_size=0.2, random_state=42)

# Convert synthetic training data to quantum features
X_train_quantum_synthetic = convert_to_quantum_features(X_train_synthetic, params)

# Train the decision tree classifier
clf.fit(X_train_quantum_synthetic, y_train_synthetic)

# Convert synthetic validation data to quantum features
X_val_quantum_synthetic = convert_to_quantum_features(X_val_synthetic, params)

# Predict on the synthetic validation set
y_pred_synthetic = clf.predict(X_val_quantum_synthetic)

# Evaluate accuracy on the synthetic validation set
accuracy_synthetic = accuracy_score(y_val_synthetic, y_pred_synthetic)
print("Accuracy on synthetic dataset:", accuracy_synthetic)


Traceback (most recent call last):
  Cell In[46], line 12
    X_train_quantum_synthetic = convert_to_quantum_features(X_train_synthetic, params)
  Cell In[45], line 31 in convert_to_quantum_features
    result = quantum_circuit(X[i], params)
  File /opt/conda/lib/python3.10/site-packages/pennylane/qnode.py:976 in __call__
    self.construct(args, kwargs)
  File /opt/conda/lib/python3.10/site-packages/pennylane/qnode.py:862 in construct
    self._qfunc_output = self.func(*args, **kwargs)
  Cell In[45], line 24 in quantum_circuit
    quantum_feature_map(x, params)
  Cell In[45], line 10 in quantum_feature_map
    qml.templates.BasicEntanglerLayers(x, wires=range(len(x)))
  File /opt/conda/lib/python3.10/site-packages/pennylane/templates/layers/basic_entangler.py:135 in __init__
    raise ValueError(
ValueError: Weights tensor must be 2-dimensional or 3-dimensional if batching; got shape (7,)

Use %tb to get the full traceback.


In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from qiskit.circuit.library import ZZFeatureMap
from qiskit.aqua import QuantumInstance
import numpy as np

# Define a function for quantum feature encoding
def quantum_encode_features(features, num_qubits):
    feature_map = ZZFeatureMap(num_qubits)
    quantum_instance = QuantumInstance(backend='statevector_simulator')
    encoded_features = np.zeros((len(features), 2**num_qubits))
    for idx, feature in enumerate(features):
        quantum_circuit = feature_map.bind_parameters(feature)
        result = quantum_instance.execute(quantum_circuit)
        encoded_features[idx] = result.get_statevector()
    return encoded_features

# Prepare dataset (dummy example)
X_train = np.random.rand(100, 5)  # 100 samples, 5 features
y_train = np.random.randint(0, 2, 100)  # Binary labels
X_test = np.random.rand(20, 5)  # 20 test samples

# Quantum feature encoding
num_qubits = 3  # Number of qubits for encoding
encoded_X_train = quantum_encode_features(X_train, num_qubits)
encoded_X_test = quantum_encode_features(X_test, num_qubits)

# Initialize classical random forest
random_forest = RandomForestClassifier(n_estimators=10)

# Train random forest
random_forest.fit(encoded_X_train, y_train)

# Predict labels
y_pred = random_forest.predict(encoded_X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Traceback (most recent call last):
  Cell In[1], line 4
    from qiskit.aqua import QuantumInstance
ModuleNotFoundError: No module named 'qiskit.aqua'

Use %tb to get the full traceback.


In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from qiskit import Aer, execute, QuantumCircuit
import numpy as np

# Define a function for quantum feature encoding
def quantum_encode_features(features, num_qubits):
    encoded_features = np.zeros((len(features), 2**num_qubits))
    for idx, feature in enumerate(features):
        qc = QuantumCircuit(num_qubits)
        for i, f in enumerate(feature):
            qc.ry(f, i)  # Encode feature using rotation gates
        backend = Aer.get_backend('statevector_simulator')
        job = execute(qc, backend)
        result = job.result().get_statevector()
        encoded_features[idx] = np.abs(result) ** 2  # Get probabilities
    return encoded_features

# Prepare dataset (dummy example)
X_train = np.random.rand(100, 5)  # 100 samples, 5 features
y_train = np.random.randint(0, 2, 100)  # Binary labels
X_test = np.random.rand(20, 5)  # 20 test samples

# Quantum feature encoding
num_qubits = 5  # Number of qubits for encoding
encoded_X_train = quantum_encode_features(X_train, num_qubits)
encoded_X_test = quantum_encode_features(X_test, num_qubits)

# Initialize classical random forest
random_forest = RandomForestClassifier(n_estimators=10)

# Train random forest
random_forest.fit(encoded_X_train, y_train)

# Predict labels
y_pred = random_forest.predict(encoded_X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Traceback (most recent call last):
  Cell In[3], line 3
    from qiskit import BasicAer, execute, QuantumCircuit
ImportError: cannot import name 'BasicAer' from 'qiskit' (/opt/conda/lib/python3.10/site-packages/qiskit/__init__.py)

Use %tb to get the full traceback.


In [6]:
! pip install qiskit-aer

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from qiskit import QuantumCircuit, transpile, Aer, execute
import numpy as np

# Define a function for quantum feature encoding
def quantum_encode_features(features, num_qubits):
    encoded_features = np.zeros((len(features), 2**num_qubits))
    for idx, feature in enumerate(features):
        qc = QuantumCircuit(num_qubits)
        for i, f in enumerate(feature):
            qc.ry(f, i)  # Encode feature using rotation gates
        backend = Aer.get_backend('statevector_simulator')
        job = execute(qc, backend)
        result = job.result().get_statevector()
        encoded_features[idx] = np.abs(result) ** 2  # Get probabilities
    return encoded_features

# Prepare dataset (dummy example)
X_train = np.random.rand(100, 5)  # 100 samples, 5 features
y_train = np.random.randint(0, 2, 100)  # Binary labels
X_test = np.random.rand(20, 5)  # 20 test samples

# Quantum feature encoding
num_qubits = 3  # Number of qubits for encoding
encoded_X_train = quantum_encode_features(X_train, num_qubits)
encoded_X_test = quantum_encode_features(X_test, num_qubits)

# Initialize classical random forest
random_forest = RandomForestClassifier(n_estimators=10)

# Train random forest
random_forest.fit(encoded_X_train, y_train)

# Predict labels
y_pred = random_forest.predict(encoded_X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Traceback (most recent call last):
  Cell In[7], line 3
    from qiskit import QuantumCircuit, transpile, Aer, execute
ImportError: cannot import name 'Aer' from 'qiskit' (/opt/conda/lib/python3.10/site-packages/qiskit/__init__.py)

Use %tb to get the full traceback.


In [8]:
! pip install qiskit-aqua

  Preparing metadata (setup.py) ... done
  Using cached fastdtw-0.3.4-cp310-cp310-linux_x86_64.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 368.2/368.2 kB 399.7 kB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... |^C
anceled
ERROR: Operation cancelled by user
